<a href="https://colab.research.google.com/github/Mario-Encinas/MachineVision_optimization/blob/main/automa_compara_algorit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:
from matplotlib import pyplot
from sklearn import datasets
import tensorflow as tf
import numpy as np    
import pandas as pd   
import time
import cv2  


#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

df = pd.DataFrame(columns = ['dataset', 'perfil_col', 'img_x', 'img_y', 'algoritmo', 'tam_entrenamiento', 'tiempo_entrenamiento', 'tam_prueba', 'tiempo_prueba', 'acierto', 'error']) 
df

,dataset,perfil_col,img_x,img_y,algoritmo,tam_entrenamiento,tiempo_entrenamiento,tam_prueba,tiempo_prueba,acierto,error


In [60]:
 #hace que la estructura de datos del array de imagen pase a tener 2 dimensiones.
 def reshape(image):
    return image.reshape(-1)

def reduce_array(imagenes, size):
  return imagenes[0:size,:,:]

def reshape_color(image):
  #nota, no estoy seguro que en realidad sean los respectivos colores, pero realmente no hace diferencia
  red = image[:,:,0]
  green = image[:,:,1]
  blue = image[:,:,2]

  red = red.reshape(-1)
  green = green.reshape(-1)
  blue = blue.reshape(-1)
  return np.concatenate((red, green, blue))

In [84]:
dataset = 'mnist' # mnist, cifar
size = 5000
perfil = 'grises'# color, grises
tam_prueba = 5000
x = 28
y = 28


if dataset == 'mnist':
  from keras.datasets import mnist
  (train_images, train_labels), (test_images, test_labels) = mnist.load_data()
else:
  from keras.datasets import cifar10
  (train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

if dataset == 'cifar' and perfil == 'grises':
  train_images_portion_grayscale = np.zeros(train_images_portion.shape[:-1])
  for i in range(train_images_portion.shape[0]): 
    train_images_portion_grayscale[i] = cv2.cvtColor(train_images_portion[i], cv2.COLOR_BGR2GRAY) 

  test_images_grayscale = np.zeros(test_images.shape[:-1])
  for i in range(test_images.shape[0]): 
    test_images_grayscale[i] = cv2.cvtColor(test_images[i], cv2.COLOR_BGR2GRAY) 
  train_images = train_images_portion_grayscale
  test_images = test_images_grayscale


train_images_portion = reduce_array(train_images, size)
train_labels_portion = train_labels[0:size]
train_images_reshape = np.array(list(map(reshape, train_images_portion)))
test_images_reshape = np.array(list(map(reshape, test_images)))

print(train_images_portion.shape)
print(train_images_reshape.shape)
print(test_images_reshape.shape)

(5000, 28, 28)
(5000, 784)
(10000, 784)


In [85]:
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

if perfil == 'color':
  dimension = 3
else:
  dimension = 1
#---

#svm_poli
classifier_svm = svm.SVC(kernel='poly',gamma='auto')

inicio = time.time()
classifier_svm.fit(train_images_reshape, train_labels_portion)
fin = time.time()
time_svm = fin-inicio

#---

#svm_linear():
"""classifier_svm_l = svm.SVC(kernel='linear',gamma='auto')

inicio = time.time()
classifier_svm_l.fit(train_images_reshape, train_labels_portion)
fin = time.time()
time_svm_l = fin-inicio

prediccion_svm_l = classifier_svm.predict(test_images_reshape)
precision_svm_l = metrics.accuracy_score(test_labels, prediccion_svm_l)
"""

#---

#tree():
classifier_tree = DecisionTreeClassifier()

inicio = time.time()
classifier_tree.fit(train_images_reshape, train_labels_portion)
fin = time.time()
time_tree = fin-inicio

#---

#forest():
classifier=RandomForestClassifier(n_estimators=100)

inicio = time.time()
classifier.fit(train_images_reshape, train_labels_portion)
fin = time.time()
time_rf = fin-inicio

#---

#nb():
classifier = GaussianNB()

inicio = time.time()
classifier.fit(train_images_reshape, train_labels_portion)
fin = time.time()
time_nb = fin-inicio

#---

#knn():
knn = KNeighborsClassifier(n_neighbors=3)

inicio = time.time()  
knn.fit(train_images_reshape, train_labels_portion)
fin = time.time()
time_knn = fin-inicio

#---

#nn
modelo_nn = tf.keras.Sequential([
  tf.keras.layers.Flatten(input_shape=(x,y,dimension)), #1 - blanco y negro
  tf.keras.layers.Dense(50, activation=tf.nn.relu),
  tf.keras.layers.Dense(50, activation=tf.nn.relu),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax) #Para redes de clasificacion
])

#Compilar el modelo
modelo_nn.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

#Entrenar
inicio = time.time()
modelo_nn.fit(train_images_portion, train_labels_portion, epochs=10)
fin = time.time()
time_nn = fin-inicio

#---

#cnn
modelo_cnn = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), input_shape=(x,y,dimension), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2), #2,2 es el tamano de la matriz

    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2), #2,2 es el tamano de la matriz

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=100, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

#Compilar el modelo
modelo_cnn.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

#Entrenar
inicio = time.time()
modelo_cnn.fit(train_images_portion, train_labels_portion, epochs=10)
fin = time.time()
time_cnn = fin-inicio

Epoch 1/10
157/157 [==============================] - 1s 3ms/step - loss: 9.0439 - accuracy: 0.6046
Epoch 2/10
157/157 [==============================] - 0s 3ms/step - loss: 1.3625 - accuracy: 0.7756
Epoch 3/10
157/157 [==============================] - 1s 4ms/step - loss: 0.8300 - accuracy: 0.8108
Epoch 4/10
157/157 [==============================] - 1s 3ms/step - loss: 0.6100 - accuracy: 0.8482
Epoch 5/10
157/157 [==============================] - 1s 3ms/step - loss: 0.4653 - accuracy: 0.8780
Epoch 6/10
157/157 [==============================] - 1s 3ms/step - loss: 0.4105 - accuracy: 0.8942
Epoch 7/10
157/157 [==============================] - 0s 3ms/step - loss: 0.3464 - accuracy: 0.9106
Epoch 8/10
157/157 [==============================] - 0s 2ms/step - loss: 0.3540 - accuracy: 0.9064
Epoch 9/10
157/157 [==============================] - 0s 3ms/step - loss: 0.2822 - accuracy: 0.9236
Epoch 10/10
157/157 [==============================] - 0s 3ms/step - loss: 0.2673 - accuracy: 0.9268

In [86]:
 inicio = time.time()
prediccion_svm = classifier_svm.predict(test_images_reshape)
precision_svm = metrics.accuracy_score(test_labels, prediccion_svm)
fin = time.time()
time_svm_p = fin-inicio


inicio = time.time()
prediccion_tree = classifier_tree.predict(test_images_reshape)
precision_tree = metrics.accuracy_score(test_labels, prediccion_tree)
fin = time.time()
time_tree_p = fin-inicio


inicio = time.time()
prediccion_rf = classifier.predict(test_images_reshape)
precision_rf = metrics.accuracy_score(test_labels, prediccion_rf)
fin = time.time()
time_rf_p = fin-inicio


inicio = time.time()
prediccion_nb = classifier.predict(test_images_reshape)
precision_nb = metrics.accuracy_score(test_labels, prediccion_nb)
fin = time.time()
time_nb_p = fin-inicio


inicio = time.time()
prediccion_knn = knn.predict(test_images_reshape)
precision_knn = metrics.accuracy_score(test_labels, prediccion_knn)
fin = time.time()
time_knn_p = fin-inicio


inicio = time.time()
_, precision_nn = modelo_nn.evaluate(test_images, test_labels)
print("Accuracy:",precision_nn, "\n")
print('Accuracy: %.2f' % (precision_nn*100))
fin = time.time()
time_nn_p = fin-inicio


inicio = time.time()
_, precision_cnn = modelo_cnn.evaluate(test_images, test_labels)
print("Accuracy:",precision_cnn, "\n")
print('Accuracy: %.2f' % (precision_cnn*100))
fin = time.time()
time_cnn_p = fin-inicio

313/313 [==============================] - 1s 2ms/step - loss: 0.9064 - accuracy: 0.8567
Accuracy: 0.8567000031471252 

Accuracy: 85.67
313/313 [==============================] - 1s 3ms/step - loss: 0.1874 - accuracy: 0.9594
Accuracy: 0.9593999981880188 

Accuracy: 95.94


In [87]:
#df = pd.DataFrame(columns = ['dataset', 'perfil_col', 'img_x', 'img_y', 'algoritmo', 'tam_entrenamiento', 'tiempo_entrenamiento', 'tam_prueba', 'tiempo_prueba', 'acierto', 'error']) 


#svm_l = [dataset, perfil, x, y, 'csv_linear',     size, time_svm_l, tam_prueba, time_svm_1_p, precision_svm_l, 1-precision_svm_l]
svm   = [dataset, perfil, x, y, 'csv_poli',       size, time_svm,   tam_prueba, time_svm_p,  precision_svm,  1-precision_svm]
dt    = [dataset, perfil, x, y, 'decision three', size, time_tree,  tam_prueba, time_tree_p, precision_tree, 1-precision_tree]
rf    = [dataset, perfil, x, y, 'random forest',  size, time_rf,    tam_prueba, time_rf_p,   precision_rf,   1-precision_rf]
nb    = [dataset, perfil, x, y, 'naive bayes',    size, time_nb,    tam_prueba, time_nb_p,   precision_nb,   1-precision_nb]
knn   = [dataset, perfil, x, y, 'knn',            size, time_knn,   tam_prueba, time_knn_p,  precision_knn,  1-precision_knn]
nn    = [dataset, perfil, x, y, 'Neural network', size, time_nn,    tam_prueba, time_nn_p,   precision_nn,   1-precision_nn]
cnn   = [dataset, perfil, x, y, 'cnn',            size, time_cnn,   tam_prueba, time_cnn_p,  precision_cnn,  1-precision_cnn] 

aux_df = pd.DataFrame([svm, dt, rf, nb, knn, nn, cnn], columns = ['dataset', 'perfil_col', 'img_x', 'img_y', 'algoritmo', 'tam_entrenamiento', 'tiempo_entrenamiento', 'tam_prueba', 'tiempo_prueba', 'acierto', 'error'])

In [88]:
df = pd.concat([df, aux_df])
df

,dataset,perfil_col,img_x,img_y,algoritmo,tam_entrenamiento,tiempo_entrenamiento,tam_prueba,tiempo_prueba,acierto,error
0,mnist,grises,32,32,csv_poli,5000,2.621097,5000,4.123532,0.9421,0.0579
1,mnist,grises,32,32,decision three,5000,0.733944,5000,0.013568,0.7784,0.2216
2,mnist,grises,32,32,random forest,5000,1.922571,5000,0.365958,0.5800,0.4200
3,mnist,grises,32,32,naive bayes,5000,0.035863,5000,0.368900,0.5800,0.4200
4,mnist,grises,32,32,knn,5000,0.000763,5000,2.616247,0.9339,0.0661
5,mnist,grises,32,32,Neural network,5000,4.494446,5000,0.859838,0.8672,0.1328
6,mnist,grises,32,32,cnn,5000,10.624361,5000,1.028390,0.9531,0.0469
0,mnist,grises,32,32,csv_poli,10000,7.242554,10000,6.658879,0.9573,0.0427
1,mnist,grises,32,32,decision three,10000,1.878141,10000,0.020263,0.8099,0.1901
2,mnist,grises,32,32,random forest,10000,4.632891,10000,0.474674,0.5738,0.4262


In [83]:
df.to_csv('raw_data.csv', index=False)
from google.colab import files
files.download("raw_data.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>